## Section 5.2: Sensitivity modelling in neural networks
### Total sensitivity modelling

In [ ]:
from deuterium import Variable, to_vec, random_symbols
import symengine as se
import numpy as np
from sys import setrecursionlimit
setrecursionlimit(1_000_000)
from skimage.measure import block_reduce
from scipy.optimize import minimize, shgo

Define some utility functions, notably the loss functions and tempered sigmoid activation functions.

In [ ]:
to_data = np.vectorize(lambda x: x.data)

def sigmoid(x, s=1, T=1, o=0):
        return (s/(1+np.exp(-T*x)))-o

def tanh(x):
    return sigmoid(x, 2, 2, 1)

bce_loss = lambda y_pred, y_true: -np.mean(np.multiply(y_true, np.log(y_pred)) + np.multiply((1 - y_true), np.log(1 - y_pred)))

Define convolution and pooling functions for the CNN.

In [ ]:
def conv2d(image, kernel):
    """Input of shape (batch_size, x, y, in_channels)
    and kernel kernel of shape (x, y, in_channels, out_channels). 
    Returns shape (batch_size, x, y, out_channels)"""
    batch_size, x_out, y_out, in_channels = image.shape
    kx, ky, _, out_channels = kernel.shape

    im_col = np.lib.stride_tricks.as_strided(
        image,
        (batch_size, x_out - kx + 1, y_out - ky + 1, kx, ky, in_channels),
        image.strides[:3] + image.strides[1:],
        writeable=False,
    )

    return np.tensordot(im_col, kernel, axes=3)


def av_pool(image):
    """Pools image of shape (batch_size, x, y, channels) down to
    (batch_size, x//2, y//2, channels)"""
    return block_reduce(image, (1, 2, 2, 1), np.mean)

Define the network architecture. This network uses TanH activations between layers and a final logistic sigmoid.

In [ ]:
SIZE=16 # Image side length (x or y)
FILTERS_1 = 8
FILTERS_2 = 4

In [ ]:
x = to_vec(np.array(random_symbols(1*SIZE*SIZE*1, "x")).reshape(1,SIZE,SIZE,1))
y = to_vec(np.array(random_symbols(1, "y")).reshape(1,1))
w1 = to_vec(np.array(random_symbols(3*3*FILTERS_1, "x")).reshape(3,3,1,FILTERS_1))
w2 = to_vec(np.array(random_symbols(3*3*FILTERS_1*FILTERS_2, "x")).reshape(3,3,FILTERS_1,FILTERS_2))

In [ ]:
all_symbols = [symb for symb in np.concatenate((x.flatten(), y.flatten(), w1.flatten(), w2.flatten()))]

Symbolically obtain the network output and gradients

In [ ]:
conv_1 = tanh(conv2d(x, w1))
pool_1 = av_pool(conv_1)
conv_2 = tanh(conv2d(pool_1, w2))
y_pred = sigmoid(conv_2.mean(axis=(1,2,3)))
loss = bce_loss(y_pred, y)

In [ ]:
loss.backward()

In [ ]:
x_grad = np.array([i.grad for i in x.flatten().tolist()])
y_grad = np.array([i.grad for i in y.flatten().tolist()])
w1_grad = np.array([i.grad for i in w1.flatten().tolist()])
w2_grad = np.array([i.grad for i in w2.flatten().tolist()])

full_grad = to_vec(np.concatenate((x_grad, y_grad, w1_grad, w2_grad)))

Obtain the sensitivity and compile the term for minimsation to calculate the Lipschitz constant.

In [ ]:
norm = np.linalg.norm(full_grad, ord=2).data #Gradient norm = Sensitivity

In [ ]:
neg_norm = se.Lambdify(to_data(all_symbols), -norm, cse=True) #norm for minimisation

TanH with constrained inputs

In [ ]:
x_bounds = [(0,1) for _ in range(len(x.flatten()))]
y_bounds = [(0,1) for _ in range(len(y.flatten()))]
w1_bounds = [(-1,1) for _ in range(len(w1.flatten()))]
w2_bounds = [(-1,1) for _ in range(len(w2.flatten()))]
all_bounds = x_bounds+y_bounds+w1_bounds+w2_bounds

x_initial = [0.5 for _ in range(len(x.flatten()))]
y_initial = [0.5 for _ in range(len(y.flatten()))]
w1_initial = [0.5 for _ in range(len(w1.flatten()))]
w2_initial = [0.5 for _ in range(len(w2.flatten()))]

all_initial = np.array(x_initial+y_initial+w1_initial+w2_initial)

The formally guaranteed way to obtain the Lipschitz constant is through the global optimum of the function. As SHGO can take a long time to converge, a faster result can be obtained using `minimize`. This is not a guaranteed minimum, however still matches the expected minimum (since we are utilising bounded activation functions).

In [ ]:
sol_m = minimize(neg_norm, bounds=all_bounds, x0=all_initial)

In [ ]:
-sol_m.fun

SHGO is provided here for reproducing the paper results.

In [ ]:
# sol_s = shgo(neg_norm, bounds=all_bounds)
# -sol_s.fun

The same activation functions with much looser bounds on the inputs should still provide the same Lipschitz constant.

In [ ]:
x_bounds = [(-100,100) for _ in range(len(x.flatten()))]
y_bounds = [(0,1) for _ in range(len(y.flatten()))]
w1_bounds = [(-100,100) for _ in range(len(w1.flatten()))]
w2_bounds = [(-100,100) for _ in range(len(w2.flatten()))]
all_bounds = x_bounds+y_bounds+w1_bounds+w2_bounds

x_initial = [0.5 for _ in range(len(x.flatten()))]
y_initial = [0.5 for _ in range(len(y.flatten()))]
w1_initial = [0.5 for _ in range(len(w1.flatten()))]
w2_initial = [0.5 for _ in range(len(w2.flatten()))]

all_initial = np.array(x_initial+y_initial+w1_initial+w2_initial)

In [ ]:
sol_m = minimize(neg_norm, bounds=all_bounds, x0=all_initial)

In [ ]:
-sol_m.fun

In [ ]:
# sol_s = shgo(neg_norm, bounds=all_bounds)
# -sol_s.fun

___

A different tempered sigmoid activation function can be used, which outputs different activation magnitudes. We use an exemplary tempered sigmoid with values $(5,5,-5)$.

In [ ]:
a = np.array([-100, -50, 0, 50, 100])

In [ ]:
sigmoid(a, 5, 5, -5)

In [ ]:
conv_1 = sigmoid(conv2d(x, w1), 5, 5, -5)
pool_1 = av_pool(conv_1)
conv_2 = sigmoid(conv2d(pool_1, w2), 5, 5, -5)
y_pred = sigmoid(conv_2.mean(axis=(1,2,3)))
loss = bce_loss(y_pred, y)

In [ ]:
loss.backward()

In [ ]:
x_grad = np.array([i.grad for i in x.flatten().tolist()])
y_grad = np.array([i.grad for i in y.flatten().tolist()])
w1_grad = np.array([i.grad for i in w1.flatten().tolist()])
w2_grad = np.array([i.grad for i in w2.flatten().tolist()])

full_grad = to_vec(np.concatenate((x_grad, y_grad, w1_grad, w2_grad)))

In [ ]:
norm = np.linalg.norm(full_grad, ord=2).data #Gradient norm = Sensitivity

In [ ]:
neg_norm = se.Lambdify(to_data(all_symbols), -norm, cse=True) #norm for minimisation

As above, we obtain the Lipschitz function with contstrained and loosely constrained inputs

In [ ]:
x_bounds = [(0,1) for _ in range(len(x.flatten()))]
y_bounds = [(0,1) for _ in range(len(y.flatten()))]
w1_bounds = [(-1,1) for _ in range(len(w1.flatten()))]
w2_bounds = [(-1,1) for _ in range(len(w2.flatten()))]
all_bounds = x_bounds+y_bounds+w1_bounds+w2_bounds

x_initial = [0.5 for _ in range(len(x.flatten()))]
y_initial = [0.5 for _ in range(len(y.flatten()))]
w1_initial = [0.5 for _ in range(len(w1.flatten()))]
w2_initial = [0.5 for _ in range(len(w2.flatten()))]

all_initial = np.array(x_initial+y_initial+w1_initial+w2_initial)

In [ ]:
sol_m = minimize(neg_norm, bounds=all_bounds, x0=all_initial)

In [ ]:
-sol_m.fun

In [ ]:
# sol_s = shgo(neg_norm, bounds=all_bounds)
# -sol_s.fun

In [ ]:
x_bounds = [(-100,100) for _ in range(len(x.flatten()))]
y_bounds = [(0,1) for _ in range(len(y.flatten()))]
w1_bounds = [(-100,100) for _ in range(len(w1.flatten()))]
w2_bounds = [(-100,100) for _ in range(len(w2.flatten()))]
all_bounds = x_bounds+y_bounds+w1_bounds+w2_bounds

x_initial = [0.5 for _ in range(len(x.flatten()))]
y_initial = [0.5 for _ in range(len(y.flatten()))]
w1_initial = [0.5 for _ in range(len(w1.flatten()))]
w2_initial = [0.5 for _ in range(len(w2.flatten()))]

all_initial = np.array(x_initial+y_initial+w1_initial+w2_initial)

In [ ]:
sol_m = minimize(neg_norm, bounds=all_bounds, x0=all_initial)

In [ ]:
-sol_m.fun

In [ ]:
# sol_s = shgo(neg_norm, bounds=all_bounds)
# -sol_s.fun